## On-demand frame batch processing using LiCSAR routines
These instructions will help you performing frame processing using licsar_framebatch. It is based on LiCS Auto-Batch routines developed by Nick Greenall. Generally speaking, you will be able to perform whole LiCSAR processing chain on the frame you identify and will see the progress in the following Google Docs spreadsheet:  
<https://docs.google.com/spreadsheets/d/1Rbt_nd5nok-UZ7dfBXFHsZ66IqozrHwxiRj-TDnVDMY/edit#gid=1398016569>
### 1. initial setup
*Please note that all instructions assume you work on CEMS sci server.*  
*You should identify the frame to be processed yourself*
First step is to load the licsar_framebatch module.
You will also need to define and create a BATCH_CACHE_DIR directory which you intend to use as a processing cache (for all frames). This should be in the group workspace. Note that all (huge) files will be generated and kept here and it is your responsibility to care of the groupwork space here (thank you for keeping things tidy).  
Finally you need to create your temporary directory.

In [ ]:
%%bash
#loading the framebatch module
module load licsar_framebatch
#defining BATCH_CACHE_DIR.. you may want to write this line to your ~/.bashrc
export BATCH_CACHE_DIR=/gws/nopw/j04/nceo_geohazards_vol1/projects/LiCS/insar_proc/my_wonderful_project
#ensuring existence of your temporary directory
mkdir /gws/nopw/j04/nceo_geohazards_vol1/insar_temp/$USER 2>/dev/null
#identify the frame name:
FRAME='124D_05278_081106'
mkdir -p $BATCH_CACHE_DIR/$FRAME

### 2. getting data from NLA (optional)
In case you want to process all data (or data older than 3 months), you should request them from NLA. You can do it using following query:  
  
*Note that it can take one full **day** or even more to finish the NLA request.  
You can see the state of it using* nla.py requests *function. If the request didn't finish at all, you may either restart the request or just go on with the images that exist so far, see further*

In [ ]:
%%bash
startdate=2014-10-01
#the db_query.list file (below) is optional but it can be later used to generate connections figure
LiCSAR_0_getFiles.py -f $FRAME -s $startdate -e $(date +%Y-%m-%d) -r -b Y -n -z $BATCH_CACHE_DIR/$FRAME/db_query.list
#check the active requests
nla.py requests
#better check of the requests status
nla_check_status.sh

### 3. Perform the frame batch processing
There is a whole-batch script **licsar_make_frame.sh** ready that will perform the whole processing chain.  
It will produce log files to ~/logs directory in your home folder (please do not change the path).  
You should run the script once. It performs the steps in a sequence (waiting to finish each one before the next is started). Since the whole processing can take few days (depending on the amount of images to process), we recommend starting the script in **screen** or **tmux**.  
  
*If you didn't perform the Step 2 - NLA request, you may start the script without the last parameter - it will process only images from the last 3 months that should be existing in the CEMS*

In [ ]:
%%bash
#the following command will perform the whole processing chain, please start it in screen or tmux:
full_scale=0 #use this if you want only last 3 months of data, i.e. not using NLA request
full_scale=1 #use this if you have done the NLA request
licsar_make_frame.sh $FRAME $full_scale

### 4. Observe the status
So, now you should see how each step fills the spreadsheet progress bar (note that the processing spreadsheet will be updated every **15 minutes**):  
<https://docs.google.com/spreadsheets/d/1Rbt_nd5nok-UZ7dfBXFHsZ66IqozrHwxiRj-TDnVDMY/edit#gid=1398016569>
   
For better understanding, the processing chain is split to following jobs:

| job type | produces.. |
|----------|-----------:|
| mk_imag | slc |
| coreg | rslc |
| mk_ifg | ifg |
| unwrap |  unwrapped ifg |
  
You may need to check the outputs from each step - the percentage of finished tasks is visible in the spreadsheet (link above). To restart the tasks you should manually copy and paste related bsub commands. It is easy when you get into it ) You may get some [more information here](licsar_framebatch_manual.ipynb) or just contact us. It will be good to investigate potential errors together..